### le rose sono rosse / le viole sono blu / prendetemi per pazzo, sì / pazzo per Gesù

In [ ]:
import funzfunz as fnz
import pickle
from bs4 import BeautifulSoup

In [ ]:
NE = 'north_east'
SW = 'south_west'
views = {
'chinatown': {SW:(45.478530132389416,9.167947775395564),NE:(45.486329944279085,9.181335323890014),},
'legit_milan': {SW:(45.43654061758556,9.1255932966908),NE:(45.52335073625751,9.25182332813564),},
'feasible_milan': {SW:(45.43654061758556,9.1255932966908),NE:(45.52335073625751,9.25182332813564),},
}

In [ ]:
n_reviews = 800
meters = 140
port = 'chinatown'

# calls payout
len(fnz.cute_types)*len(fnz.get_radars(meters, **view[port]))

##### API places

In [ ]:
# pickle.dump(fnz.sample_viewport(meters, view[port]), open(port+"_places_db.pkl", "wb")

In [ ]:
places_db = pickle.load(open(port+"_places_db.pkl", "rb")) # list of dict-place

##### purify dataset

In [ ]:
#pickle.dump(purify_data(places_db), open(port+"_data_low.pkl", "wb"))

In [ ]:
data_low = pickle.load(open(port+"_data_low.pkl", "rb"))

##### scraping reviews

In [ ]:
# pickle.dump(fnz.scrape_reviews([i['place_id'] for i in data], n_reviews), open(port+"_reviews_db.pkl", "wb"))

In [ ]:
reviews_db = pickle.load(open(port+"_reviews_db.pkl", "rb"))  # dict of list of htmls <- to BeautifulSoup(htmls)

##### upgrade data

In [ ]:
#pickle.dump(fnz.enrich_data(data_low, reviews_db), open(port+"_data_high.pkl", "wb"))

In [ ]:
data_high = pickle.load(open("data.pkl", "rb"))  # beautiful json

In [ ]:
from collections import Counter
prz = []
for i in places_db:
    if 'price_level' in i:
        prz.append(i['price_level'])
print(len(prz))
print(Counter(prz))